In [15]:
import numpy as np 
import pandas as pd
import re
from nltk.stem.porter import PorterStemmer
import string
from nltk.tokenize import word_tokenize
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.svm import LinearSVC

In [16]:
from google.colab import files
uploaded = files.upload()

Saving sample_submission.csv to sample_submission (1).csv
Saving test.csv to test (1).csv
Saving train.csv to train (1).csv


In [18]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [19]:
df_test.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [20]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [21]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3263 entries, 0 to 3262
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        3263 non-null   int64 
 1   keyword   3237 non-null   object
 2   location  2158 non-null   object
 3   text      3263 non-null   object
dtypes: int64(1), object(3)
memory usage: 102.1+ KB


In [22]:
#Save the 'Id' column
train_ID = df_train['id']
test_ID = df_test['id']

#Now drop the  'Id' colum since it's unnecessary for  the prediction process.
df_train.drop("id", axis = 1, inplace = True)
df_test.drop("id", axis = 1, inplace = True)

In [23]:
print(df_train.isnull().sum())
df_test.isnull().sum()

keyword       61
location    2533
text           0
target         0
dtype: int64


keyword       26
location    1105
text           0
dtype: int64

In [24]:
#Replace NaN values by None
df_train['keyword'] = df_train['keyword'].fillna('None')
df_test['keyword'] = df_test['keyword'].fillna('None')
df_train['location'] = df_train['location'].fillna('None')
df_test['location'] = df_test['location'].fillna('None')

In [25]:
df_train["text"]= df_train["keyword"] + " " + df_train["location"] + " "+df_train["text"]
df_test["text"]= df_test["keyword"] + " " + df_test["location"] + " "+df_test["text"]

df_train=df_train.drop("keyword",axis=1)
df_train=df_train.drop("location",axis=1)

df_test=df_test.drop("keyword",axis=1)
df_test=df_test.drop("location",axis=1)

In [27]:
#Remove redundant samples
df_train=df_train.drop_duplicates(subset=['text', 'target'], keep='first')
df_train.shape

(7561, 2)

In [35]:
import nltk
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
def clean(sen):
    sentence = re.sub("http[s]*://[^\s]+"," ",sen)
    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)
    
    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)
    
    sentence = str(sentence).lower()
    sentence = sentence.replace(",000,000", "m").replace(",000", "k").replace("′", "'").replace("’", "'")\
                           .replace("won't", "will not").replace("cannot", "can not").replace("can't", "can not")\
                           .replace("n't", " not").replace("what's", "what is").replace("it's", "it is")\
                           .replace("'ve", " have").replace("i'm", "i am").replace("'re", " are")\
                           .replace("he's", "he is").replace("she's", "she is").replace("'s", " own")\
                           .replace("%", " percent ").replace("₹", " rupee ").replace("$", " dollar ")\
                           .replace("€", " euro ").replace("'ll", " will")
    sentence = re.sub(r"([0-9]+)000000", r"\1m", sentence)
    sentence = re.sub(r"([0-9]+)000", r"\1k", sentence)
    sentence = re.sub(r"\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s\/]*))*", "", sentence)
    sentence = sentence.replace("_", " ")
    
    return sentence

pstem = PorterStemmer()
def clean_text(text):
    text= text.lower()
    text= re.sub('[0-9]', '', text)
    text  = "".join([char for char in text if char not in string.punctuation])
    tokens = word_tokenize(text)
    tokens=[pstem.stem(word) for word in tokens]
    tokens=[word for word in tokens if word not in stopwords.words('english')]
    text = ' '.join(tokens)
    return text

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [36]:
nltk.download('punkt')
df_train['text'] = df_train['text'].apply(lambda s : clean(s))
df_test['text'] = df_test['text'].apply(lambda s : clean(s))
df_train['text'] = df_train['text'].apply(lambda s : clean_text(s))
df_test['text'] = df_test['text'].apply(lambda s : clean_text(s))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [37]:
X_train = df_train.loc[:,df_train.columns != 'target'] 
y_train = df_train['target']

In [38]:

preprocessor = ColumnTransformer(
     transformers=[
         ('text', TfidfVectorizer(), 'text'),]
    ,)

text_clf = Pipeline([('preprocessor', preprocessor),
                     ('clf', LinearSVC(loss='hinge'),),
])

# Feed the training data through the pipeline
text_clf.fit(X_train, y_train) 

Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('text',
                                                  TfidfVectorizer(analyzer='word',
                                                                  binary=False,
                                                                  decode_error='strict',
                                                                  dtype=<class 'numpy.float64'>,
                                                                  encoding='utf-8',
                                                                  input='content',
                                                                  lowercase=True,
                                                                  max_df=1.0,
                           

In [39]:
predictions = text_clf.predict(df_test)
output = pd.DataFrame({'id': test_ID,
                       'target': predictions})

output.to_csv('submission.csv', index=False)

In [40]:
output.head()

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1
